In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import os.path
from os import listdir

%run ./classifica_serie_a.ipynb
%run ../../UtilityPython.ipynb

c:\Users\pagan133\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


./classifica-serie-a/2019-01-01_2019-08-26_0.csv
./classifica-serie-a/2019-08-27_2019-09-01_1.csv
./classifica-serie-a/2019-09-02_2019-09-16_2.csv
./classifica-serie-a/2019-09-17_2019-09-22_3.csv
./classifica-serie-a/2019-09-23_2019-09-26_4.csv
./classifica-serie-a/2019-09-27_2019-09-30_5.csv
./classifica-serie-a/2019-10-01_2019-10-06_6.csv
./classifica-serie-a/2019-10-07_2019-10-21_7.csv
./classifica-serie-a/2019-10-22_2019-10-27_8.csv
./classifica-serie-a/2019-10-28_2019-10-31_9.csv
./classifica-serie-a/2019-11-01_2019-11-04_10.csv
./classifica-serie-a/2019-11-05_2019-11-10_11.csv
./classifica-serie-a/2019-11-11_2019-11-25_12.csv
./classifica-serie-a/2019-11-26_2019-12-02_13.csv
./classifica-serie-a/2019-12-03_2019-12-08_15.csv
./classifica-serie-a/2019-12-09_2019-12-16_15.csv
./classifica-serie-a/2019-12-17_2019-12-22_16.csv
./classifica-serie-a/2019-12-23_2020-01-06_17.csv
./classifica-serie-a/2020-01-07_2020-01-13_18.csv
./classifica-serie-a/2020-01-13_2020-01-20_19.csv
./classifi

Exception: File `'../../UtilityPython.ipynb.py'` not found.

In [7]:
def read_more_file(path):
    filepaths = [f for f in listdir(path) if f.endswith('.csv')]
    print(filepaths)
    filepaths=list(map(lambda x : path + x,filepaths))
    print(filepaths)

    return pd.concat(map(lambda x : pd.read_csv(x, sep='~', keep_default_na=False, na_values=['_'],
                                                error_bad_lines=False), filepaths))


#Creo un nuovo dataframe con: (id, pro, data, max us, hour, type) 
def reshape_df(df, start_date, end_date):
    sdate = datetime.strptime(start_date, '%d_%m_%Y')
    edate = datetime.strptime(end_date, '%d_%m_%Y')
    
    delta = edate - sdate       # as timedelta
    final = pd.DataFrame(columns = ['ID', 'PROVINCIA','DATA', 'CAPACITA', 'SOGLIA GIALLA', 'SOGLIA ROSSA', 
                                    'MAX_DS', 'INTERVALLO_MAX_DS', 'TIPO_GIORNO_DS'])
    
    for i in range(delta.days + 1):
        day = sdate + timedelta(days=i)
        
        tmp = df.filter(regex= datetime.strftime(day, '%d_%m_%Y')).rename(
            columns={'MAX DS ' + datetime.strftime(day, '%d_%m_%Y'):'MAX_DS',
                     'INTERVALLO MAX DS ' + datetime.strftime(day, '%d_%m_%Y'):'INTERVALLO_MAX_DS',
                     'TIPO GIORNO DS '+ datetime.strftime(day, '%d_%m_%Y'):'TIPO_GIORNO_DS'})
        df_anag = df[['ID', 'PROVINCIA', 'CAPACITA', 'SOGLIA GIALLA', 'SOGLIA ROSSA',]]
        tmp1 = pd.concat([df_anag, tmp], axis=1, sort=False)
        tmp1.dropna(inplace=True)
        tmp1['DATA'] = day
        final=final.append(tmp1)
        
    return final

#Setta il flag sulla logica dei team-partita
def set_game_flag(team1, team2, s_d, squadra):
    value = 0
    if team1==squadra or team2==squadra:
        value = 0.25 if s_d=='SKY' else 1
    return value

#Definisce il dataframe che associa le informazioni tra la partita e quale squadra ha giocato
def calculate_match_vector(df):
    match = pd.DataFrame(columns=['DATA', 'GAME_J', 'GAME_I', 'GAME_M', 'GAME_R', 'GAME_N'])
    
    for index, row in df.iterrows():
        if row['data'] not in match['DATA'].unique():
            match = match.append(pd.Series([row['data'], 0, 0, 0, 0, 0], index=match.columns ),
                                 ignore_index=True)

        match.loc[(match['DATA']==row['data']) & (match['GAME_J']==0),'GAME_J']=set_game_flag(
            row['casa'], row['trasferta'] , row['diretta'], 'Juventus')
        match.loc[(match['DATA']==row['data']) & (match['GAME_I']==0),'GAME_I']=set_game_flag(
            row['casa'], row['trasferta'] , row['diretta'], 'Inter')
        match.loc[(match['DATA']==row['data']) & (match['GAME_M']==0),'GAME_M']=set_game_flag(
            row['casa'], row['trasferta'] , row['diretta'], 'Milan')
        match.loc[(match['DATA']==row['data']) & (match['GAME_R']==0),'GAME_R']=set_game_flag(
            row['casa'], row['trasferta'] , row['diretta'], 'Roma')
        match.loc[(match['DATA']==row['data']) & (match['GAME_N']==0),'GAME_N']=set_game_flag(
            row['casa'], row['trasferta'] , row['diretta'], 'Napoli')
    return match

def kpi_partita(data, juv, inter, mil, rm, na, g_j, g_i, g_m, g_r, g_n):
    data_cal=data[:10]
    '''
    return (juv*g_j*get_posizione_classifica(data_cal, "Juventus")
            + inter*g_i*get_posizione_classifica(data_cal, "Inter") 
            + mil*g_m*get_posizione_classifica(data_cal, "Milan") 
            + rm*g_r*get_posizione_classifica(data_cal, "Roma") 
            + na*g_n*get_posizione_classifica(data_cal, "Napoli"))#*match_hour(data)
            '''
    m_j = juv*g_j if g_j!=0 else juv*0.05 if (g_i!=0) | (g_m!=0) | (g_r!=0) | (g_n!=0) else 0
    i_i = inter*g_i if g_i!=0 else inter*0.05 if (g_j!=0) | (g_m!=0) | (g_r!=0) | (g_n!=0) else 0
    m_m = mil*g_m if g_m!=0 else mil*0.05 if (g_j!=0) | (g_i!=0) | (g_r!=0) | (g_n!=0) else 0
    m_r = rm*g_r if g_r!=0 else rm*0.05 if (g_j!=0) | (g_i!=0) | (g_m!=0) | (g_n!=0) else 0
    m_n = na*g_n if g_n!=0 else na*0.05 if (g_j!=0) | (g_i!=0) | (g_m!=0) | (g_r!=0)  else 0
    return (m_j + i_i + m_m + m_r + m_n)#*match_hour(data)

def match_hour(data):
    data = datetime.strptime(data, "%Y-%m-%d %H:%M:%S")
    return 2 if data.hour==21 else 0.5 if data.hour==18 else 0.6

In [8]:
#df: dataset del traffico
#kpi: dataset dei kpi delle partite
#prv: dataset con l'associazione regione-> provincia
def pre_processing(df, kpi, prv):
    #Eliminiamo gli apparati che hanno misure non valide
    df=df[df['FLAG MISURA MANCANTE']!='Y']
    
    #Filtriamo le sole features che ci interessano, la misura dei picchi di traffico D e 
    #le anagrafiche che definiscono l'apparato con la provincia di appartenenza
    df_new1 = df.filter(regex='MAX DS ')
    df_new2 = df.filter(regex='TIPO GIORNO DS ')
    df_anag = df[['APPARATO', 'CATEGORIA','ID_MISURE_ATTESTAZIONE', 'PROVINCIA', 'CAPACITA', 'SOGLIA GIALLA',
                  'SOGLIA ROSSA', 'LINEE RES', 'LINEE BUS', 'LINEE WHO']]
    result = pd.concat([df_anag, df_new1, df_new2], axis=1, sort=False)
    
    #Eliminiamo i dati di cui non abbiamo anagrafica (provincia o linee null) se presenti
    result['LINEE RES'] = pd.to_numeric(result['LINEE RES'])
    result['LINEE BUS'] = pd.to_numeric(result['LINEE BUS'])
    result['LINEE WHO'] = pd.to_numeric(result['LINEE WHO'])
    result.dropna(subset = ['LINEE RES', 'LINEE BUS', 'LINEE WHO', 'PROVINCIA'],inplace=True)

    #Calcolo le percentuali per ottenere ogni apparato quante tipologie di linee possiede
    result['LINEE RES PERC']=result['LINEE RES']/(result['LINEE RES']+result['LINEE BUS']+result['LINEE WHO'])
    result['LINEE BUS PERC']=result['LINEE BUS']/(result['LINEE RES']+result['LINEE BUS']+result['LINEE WHO'])
    result['LINEE WHO PERC']=result['LINEE WHO']/(result['LINEE RES']+result['LINEE BUS']+result['LINEE WHO'])
    result.drop(['LINEE RES', 'LINEE BUS', 'LINEE WHO'],axis=1,inplace=True)
    
    #Filtriamo le linee che hanno percentuale di residenziali > 0.75
    result_f=result[result['LINEE RES PERC']>=0.75]
    result_f.drop(['LINEE RES PERC', 'LINEE BUS PERC', 'LINEE WHO PERC'],axis=1,inplace=True)
    
    #Creiamo ID
    result_f['ID']=result_f['APPARATO']+result_f['CATEGORIA']+result_f['ID_MISURE_ATTESTAZIONE']
    result_f.drop(['APPARATO', 'CATEGORIA', 'ID_MISURE_ATTESTAZIONE'],axis=1,inplace=True)
    cols = ['ID']  + [col for col in result_f if col != 'ID']
    result_f = result_f[cols]
    
    #Prendiamo gli apparati che risultano presenti su tutte le settimane contenute nel dataframe 
    tmp=result_f.groupby(['ID','PROVINCIA']).size().to_frame('number_of_week').sort_values(
        ['number_of_week'], ascending=[False])
    num_of_weeks = tmp.number_of_week.max()
    join=pd.merge(result_f, tmp, on='ID', how='outer')
    result_f=join[join.number_of_week==num_of_weeks]
    result_f.drop(['number_of_week'],axis=1,inplace=True)
    
    #Richiamiamo la funzione reshape_df per splittare le colonne in righe
    finale=reshape_df(result_f, result_f.columns[5].split(' ')[-1], result_f.columns[-1].split(' ')[-1])
    finale['DATA']=finale['DATA'].dt.strftime('%d-%m-%Y')
    #Scartiamo i valori zero di DS
    finale=finale[finale.MAX_DS!='0']
    
    #Conversione tipi delle colonne
    finale['MAX_DS'] = finale['MAX_DS'].str.replace(',', '.').astype(float)
    finale['SOGLIA GIALLA'] = finale['SOGLIA GIALLA'].str.replace(',', '.').astype(float)
    finale['SOGLIA ROSSA'] = finale['SOGLIA ROSSA'].str.replace(',', '.').astype(float)
    finale['CAPACITA'] = finale['CAPACITA'].str.replace(',', '.').astype(float)
    
    #Ricaviamo le informazioni dalle colonne
    finale.loc[finale['SOGLIA GIALLA']<finale['MAX_DS'], 'SOPRA_SOGLIA_GIALLA'] =1
    finale.loc[finale['SOGLIA ROSSA']<finale['MAX_DS'], 'SOPRA_SOGLIA_ROSSA'] =1
    finale.loc[finale['CAPACITA']<finale['MAX_DS'], 'SOPRA_CAPACITA'] =1
    finale.fillna({'SOPRA_SOGLIA_GIALLA':0, 'SOPRA_SOGLIA_ROSSA':0, 'SOPRA_CAPACITA':0}, inplace=True)
    finale.drop(['CAPACITA', 'SOGLIA GIALLA', 'SOGLIA ROSSA'], axis=1, inplace=True)
    
    finale.loc[:,'DATA'] = pd.to_datetime(finale.DATA + ' ' + finale.INTERVALLO_MAX_DS,
                                          format="%d-%m-%Y %H:%M")

    traf_f=finale.groupby([pd.Grouper(key='DATA', freq='3H'), 'PROVINCIA', 'TIPO_GIORNO_DS']).agg({'MAX_DS': 'sum', 
                                             'SOPRA_SOGLIA_GIALLA' : 'sum',
                                             'SOPRA_SOGLIA_ROSSA' : 'sum',
                                             'SOPRA_CAPACITA' : 'sum'}).reset_index()
    tmp = finale.groupby([pd.Grouper(key='DATA', freq='3H'), 'PROVINCIA', 'TIPO_GIORNO_DS']).size().to_frame(
        'N_APPARATI').reset_index()
    traf_f=pd.merge(traf_f, tmp, on=['DATA', 'PROVINCIA', 'TIPO_GIORNO_DS'], how='outer').reset_index(
        drop=True)

    #Convertiamo il tipo di giorno FER o FES in 0 o 1
    traf_f.replace({'TIPO_GIORNO_DS': {'FER': 0, 'FES': 1}}, inplace=True)

    #Aggiungiamo la regione
    traf_f=pd.merge(traf_f, prv, on=['PROVINCIA'], how='left').reset_index(drop=True)
    
    #aggiungiamo il kpi
    traf_f=pd.merge(traf_f, kpi, on=['REGIONE', 'DATA'], how='left').reset_index(drop=True)
    
    #Dove il KPI è null vuol dire che non c'è l'evento. Lo impostiamo a 0
    traf_f=traf_f.fillna(0)
    
    if os.path.exists('./datasets/to_train.csv'):
        traf_f.to_csv (r'./datasets/to_train.csv', index = None, header=False, mode='a')
    else:
        traf_f.to_csv (r'./datasets/to_train.csv', index = None, header=True)


In [9]:
def anag():
    #Leggiamo il calendario della SERIE A (fix su NA)
    cal = pd.read_csv('./datasets/calendario_completo_processed.csv')
    cal.loc[cal['casa_prov']=='NP','casa_prov']='NA'
    cal.loc[cal['trasferta_prov']=='NP','trasferta_prov']='NA'

    #Calcoliamo le informazioni sul match
    match=calculate_match_vector(cal)
    
    #Anagrafica prelevata da https://www.istat.it/it/archivio/6789
    prv = pd.read_csv('./datasets/Elenco-comuni-italiani.csv', sep=';', encoding='latin-1',
                      keep_default_na=False, na_values=['_'])
    prv=prv.drop_duplicates(['Denominazione regione', 'Sigla automobilistica'])[
    ['Denominazione regione', 'Sigla automobilistica']]
    prv.rename(columns={'Denominazione regione':'REGIONE', 
                        'Sigla automobilistica':'PROVINCIA'
                       }, 
                     inplace=True)

    #Aggiungiamo le province mancanti
    listOfSeries = [pd.Series(['Sardegna', 'OT'], index=prv.columns ) ,
                    pd.Series(['Sardegna', 'OG'], index=prv.columns ) ,
                    pd.Series(['Sardegna', 'CI'], index=prv.columns ),
                    pd.Series(['Emilia-Romagna', 'SM'], index=prv.columns ), #Repubblica di San Marino inserita per comodità in Emilia
                    pd.Series(['Sardegna', 'VS'], index=prv.columns ) ]
    prv = prv.append(listOfSeries , ignore_index=True)
    prv.reset_index(drop=True)
    
    #Anagrafica dei tifosi basata sui dati https://forum.termometropolitico.it/170882-percentuale-di-tifosi-per-squadra-italia-regione-per-regione.html
    dictionary = {'REGIONE' : pd.Series(['Piemonte', 'Lombardia', "Valle d'Aosta/Vallée d'Aoste",
                                         'Trentino-Alto Adige/Südtirol', 'Veneto', 'Friuli-Venezia Giulia',
                                         'Liguria', 'Emilia-Romagna', 'Toscana', 'Umbria', 'Marche',
                                         'Lazio', 'Abruzzo', 'Molise', 'Campania', 'Puglia', 'Basilicata',
                                         'Calabria', 'Sicilia', 'Sardegna']),
                  'JUVENTUS' : pd.Series([34.1, 22.5, 34.1, 33.4, 28.1, 24.9, 20.4, 24.2, 23.2, 23, 34,
                                          16.3, 33.5, 32.9, 17.1, 28.8, 31.3, 29.6, 33.8, 23.4]),
                  'INTER' : pd.Series([11.2, 26.9, 11.2, 17.1, 16.9, 12.8, 9.2, 15, 12.4, 16.1, 17.9, 6,
                                       19.8, 19, 10, 14.4, 26, 16.4, 17.2, 13.2] ),
                  'MILAN' : pd.Series([26.1, 10.9, 26.1, 24.3, 22.5, 23.5, 10.5, 18.7, 16.5, 18, 18.4, 6.5,
                                       19.2, 22.2, 9.4, 19.9, 15.1, 14.1, 19.1, 12.6]), 
                  'ROMA' : pd.Series([1.5, np.NaN, 1.5, 3.9, np.NaN, np.NaN, 1.7, np.NaN, 2.5, 4.4, 3.3,
                                      35.1, 5.8, 4.2, 2.9, 4.4, 3.8, 4.4, 3.8, 3.5]),
                  'NAPOLI' : pd.Series([3.3, 4.6, 3.3, 4.5, 3.6, 2.8, 3.4, 3.3, 4.5, np.NaN, np.NaN, 4, 3.2,
                                        7.9, 46.7, 4.7, 8.6, 4.5, 3.2, 2.7])}

    anag_tifosi = pd.DataFrame(dictionary)
    anag_tifosi=anag_tifosi.fillna(0)
    anag_tifosi.JUVENTUS=anag_tifosi.JUVENTUS/100
    anag_tifosi.INTER=anag_tifosi.INTER/100
    anag_tifosi.MILAN=anag_tifosi.MILAN/100
    anag_tifosi.ROMA=anag_tifosi.ROMA/100
    anag_tifosi.NAPOLI=anag_tifosi.NAPOLI/100
    
    anag_tifosi['key']=1
    match['key']=1
    #Join tra l'anagrafica dei tifosi e quella dei match
    df_anag = pd.merge(anag_tifosi, match, on='key')
    
    #Calcoliamo il valore di kpi per ogni partita e per ogni regione
    kpi = pd.DataFrame(columns=['REGIONE', 'DATA', 'KPI'])

    for index, row in df_anag.iterrows():
        kpi = kpi.append(pd.Series([row['REGIONE'], row['DATA'], kpi_partita(row['DATA'],
                                                                             row['JUVENTUS'], 
                                                                             row['INTER'],
                                                                             row['MILAN'],
                                                                             row['ROMA'], 
                                                                             row['NAPOLI'],
                                                                             row['GAME_J'],
                                                                             row['GAME_I'],
                                                                             row['GAME_M'], 
                                                                             row['GAME_R'], 
                                                                             row['GAME_N'])]
                                   , index=kpi.columns), ignore_index=True)
        
    kpi['DATA']=pd.to_datetime(kpi['DATA'])
    kpi['DATA'] = pd.DatetimeIndex(kpi['DATA']) + timedelta(hours=1)

    #Raggruppiamo i kpi su 3H 
    kpi_tmp=kpi.groupby(['REGIONE',pd.Grouper(key='DATA', freq='3H'), 'KPI']).size().to_frame('size').reset_index()
    #I KPI si devono associare alle 3H precedenti, così da permettere la predizione sull'evento futuro
    kpi_tmp['DATA'] = pd.DatetimeIndex(kpi_tmp['DATA']) - timedelta(hours=3)
    kpi_tmp.drop('size',axis=1,inplace=True)
    
    return kpi_tmp, prv

In [10]:
if os.path.exists('./datasets/to_train.csv'):
    os.remove('./datasets/to_train.csv')
    
kpi, prv = anag()

#df=read_more_file('./Datasets/input/08/')
#pre_processing(df, kpi, prv)
#df=read_more_file('./Datasets/input/09/')
#pre_processing(df, kpi, prv)
df=read_more_file('./Datasets/input/10/')
pre_processing(df, kpi, prv)
df=read_more_file('./Datasets/input/11/')
pre_processing(df, kpi, prv)
df=read_more_file('./Datasets/input/12/')
pre_processing(df, kpi, prv)
df=read_more_file('./Datasets/input/20_01/')
pre_processing(df, kpi, prv)
#df=read_more_file('./Datasets/input/20_02/')
#pre_processing(df, kpi, prv)

['201941_Report_settimanale_DSLAM-OLT_07_10_2019_13_10_2019.csv', '201942_Report_settimanale_DSLAM-OLT_14_10_2019_20_10_2019.csv', '201943_Report_settimanale_DSLAM-OLT_21_10_2019_27_10_2019.csv', '201944_Report_settimanale_DSLAM-OLT_28_10_2019_03_11_2019.csv']
['./Datasets/input/10/201941_Report_settimanale_DSLAM-OLT_07_10_2019_13_10_2019.csv', './Datasets/input/10/201942_Report_settimanale_DSLAM-OLT_14_10_2019_20_10_2019.csv', './Datasets/input/10/201943_Report_settimanale_DSLAM-OLT_21_10_2019_27_10_2019.csv', './Datasets/input/10/201944_Report_settimanale_DSLAM-OLT_28_10_2019_03_11_2019.csv']


C:\Users\ant.pagano\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



['201945_Report_settimanale_DSLAM-OLT_04_11_2019_10_11_2019.csv', '201946_Report_settimanale_DSLAM-OLT_11_11_2019_17_11_2019.csv', '201947_Report_settimanale_DSLAM-OLT_18_11_2019_24_11_2019.csv', '201948_Report_settimanale_DSLAM-OLT_25_11_2019_01_12_2019.csv']
['./Datasets/input/11/201945_Report_settimanale_DSLAM-OLT_04_11_2019_10_11_2019.csv', './Datasets/input/11/201946_Report_settimanale_DSLAM-OLT_11_11_2019_17_11_2019.csv', './Datasets/input/11/201947_Report_settimanale_DSLAM-OLT_18_11_2019_24_11_2019.csv', './Datasets/input/11/201948_Report_settimanale_DSLAM-OLT_25_11_2019_01_12_2019.csv']
['201949_Report_settimanale_DSLAM-OLT_02_12_2019_08_12_2019.csv', '201950_Report_settimanale_DSLAM-OLT_09_12_2019_15_12_2019.csv', '201951_Report_settimanale_DSLAM-OLT_16_12_2019_22_12_2019.csv', '201952_Report_settimanale_DSLAM-OLT_23_12_2019_29_12_2019.csv']
['./Datasets/input/12/201949_Report_settimanale_DSLAM-OLT_02_12_2019_08_12_2019.csv', './Datasets/input/12/201950_Report_settimanale_DSLA